In [2]:
import pandas as pd

header = [
    "transaction_id", "price", "date_of_transfer", "postcode", "property_type",
    "new_build_flag", "tenure_type", "paon", "saon", "street", "locality",
    "town_city", "district", "county", "ppd_category_type", "record_status"
]

file_path = "propertyprice/pp-2023.csv"

# 读取数据
df = pd.read_csv(file_path, header=None, names=header)

# 保存带表头的新文件（可覆盖原文件或另存为新文件）
df.to_csv(file_path, index=False)

# 筛选county为伦敦（London）的数据
df_london = df[df['county'].str.lower() == 'greater london']


/var/folders/3f/_f93wsjd5qj0zgvg5mxm07yh0000gn/T/ipykernel_72657/2102968695.py:12: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, header=None, names=header)


In [3]:
df_london.head()
len(df_london)


94835

In [4]:
keep_cols = [
    "transaction_id",     # 唯一交易编号
    "price",              # 成交价格
    "date_of_transfer",   # 交易时间
    "postcode",           # 用于后续合并坐标
    "property_type",      # 房产类型（如Detached/Flat等）
    "new_build_flag",     # 是否为新建
    "tenure_type",        # 持有方式（Freehold/Leasehold）
    "county",             # 地理级别信息，用于区分region
    "ppd_category_type",  # A为标准交易，B为其他（如公司转让）
    "record_status"       # 数据修订状态（如添加/修改/删除）
]
df_clean = df_london[keep_cols]
print(df_clean)

df_clean.to_csv("propertyprice/pp-2023-london.csv", index=False)

                                transaction_id   price  date_of_transfer  \
216     {0E082196-DDB8-5C09-E063-4704A8C0A10E}  425000  2023-08-03 00:00   
217     {0E082196-DDBA-5C09-E063-4704A8C0A10E}  410000  2023-11-03 00:00   
218     {0E082196-DDBB-5C09-E063-4704A8C0A10E}  692000  2023-08-18 00:00   
219     {0E082196-DDBC-5C09-E063-4704A8C0A10E}  425000  2023-10-13 00:00   
220     {0E082196-DDBD-5C09-E063-4704A8C0A10E}  460000  2023-11-02 00:00   
...                                        ...     ...               ...   
850199  {06C9F487-5325-9388-E063-4804A8C0BD98}  325000  2023-08-18 00:00   
850200  {06C9F487-5326-9388-E063-4804A8C0BD98}  532500  2023-08-09 00:00   
850201  {06C9F487-5327-9388-E063-4804A8C0BD98}  317000  2023-09-05 00:00   
850202  {06C9F487-5328-9388-E063-4804A8C0BD98}  630000  2023-08-11 00:00   
850203  {06C9F487-532A-9388-E063-4804A8C0BD98}  200000  2023-08-11 00:00   

       postcode property_type new_build_flag tenure_type          county  \
216      E7

与postcode映射


In [6]:

# 读取两个文件
df_pp = pd.read_csv("propertyprice/pp-2023-london.csv")
df_latlng = pd.read_csv("propertyprice/LAT_LONGT_P_london.csv")

# 标准化邮编格式（去空格并大写，推荐）
df_pp['postcode'] = df_pp['postcode'].str.replace(' ', '').str.upper()
df_latlng['PCDS'] = df_latlng['PCDS'].str.replace(' ', '').str.upper()

# 以postcode和PCDS为键，左连接
df_merged = pd.merge(df_pp, df_latlng, left_on='postcode', right_on='PCDS', how='left')

# 保存合并后的结果
df_merged.to_csv("pp-2023-london-merged.csv", index=False)

df_2023 = pd.read_csv("pp-2023-london-merged.csv")
df_2023.head()



,transaction_id,price,date_of_transfer,postcode,property_type,new_build_flag,tenure_type,county,ppd_category_type,record_status,PCDS,LAT,LONG,LSOA11,OA11,IMD,CTRY,RGN,OSLAUA
0,{0E082196-DDB8-5C09-E063-4704A8C0A10E},425000,2023-08-03 00:00,E78HP,T,N,F,GREATER LONDON,A,A,E78HP,51.54092,0.030448,E01003573,E00018026,9549.0,E92000001,E12000007,E09000025
1,{0E082196-DDBA-5C09-E063-4704A8C0A10E},410000,2023-11-03 00:00,RM39RS,S,N,F,GREATER LONDON,A,A,RM39RS,51.61081,0.235657,E01002279,E00011373,10596.0,E92000001,E12000007,E09000016
2,{0E082196-DDBB-5C09-E063-4704A8C0A10E},692000,2023-08-18 00:00,E152BA,T,N,F,GREATER LONDON,A,A,E152BA,51.55255,-0.002701,E01004337,E00021832,10118.0,E92000001,E12000007,E09000031
3,{0E082196-DDBC-5C09-E063-4704A8C0A10E},425000,2023-10-13 00:00,RM79JB,T,N,F,GREATER LONDON,A,A,RM79JB,51.57322,0.161352,E01002246,E00011233,19248.0,E92000001,E12000007,E09000016
4,{0E082196-DDBD-5C09-E063-4704A8C0A10E},460000,2023-11-02 00:00,E130AG,T,N,F,GREATER LONDON,A,A,E130AG,51.53024,0.020861,E01003595,E00175076,8140.0,E92000001,E12000007,E09000025


In [7]:
len(df_2023)

94835

与EPC合并

去重

In [8]:
import pandas as pd

# —— Step 1: 读取数据 ——
pp_df = pd.read_csv("pp-2023-london-merged.csv")
epc_df = pd.read_csv("EPCdata/merged_london_epc_clean.csv")

# —— Step 2: 筛选2023年的EPC数据 ——
epc_df["LODGEMENT_DATE"] = pd.to_datetime(epc_df["LODGEMENT_DATE"], errors="coerce")
epc_2023 = epc_df[epc_df["LODGEMENT_DATE"].dt.year == 2023]

# —— Step 3: 清洗 Postcode 字段 ——
pp_df["postcode_clean"] = pp_df["postcode"].str.replace(" ", "").str.upper()
epc_2023["POSTCODE_clean"] = epc_2023["POSTCODE"].str.replace(" ", "").str.upper()

# —— Step 4: 创建 EPC 对应的 PPD-style property_type —— #
def map_property_type_epc(row):
    if row["PROPERTY_TYPE"] == "House":
        bf = str(row["BUILT_FORM"])
        if "Detached" in bf:
            return "D"
        elif "Semi" in bf:
            return "S"
        elif "Terrace" in bf:
            return "T"
        else:
            return "O"
    elif row["PROPERTY_TYPE"] in ["Flat", "Maisonette"]:
        return "F"
    else:
        return "O"

epc_2023["ppd_property_type"] = epc_2023.apply(map_property_type_epc, axis=1)

# —— Step 5: 重命名 pp_df 的 property_type 字段以匹配 —— #
pp_df["ppd_property_type"] = pp_df["property_type"]

# —— Step 6: 对 EPC 数据进行 postcode + property_type 聚合 —— #
epc_grouped = epc_2023.groupby(["POSTCODE_clean", "ppd_property_type"]).agg({
    "TOTAL_FLOOR_AREA": "median",
    "CONSTRUCTION_AGE_BAND": "min",
    "NUMBER_HABITABLE_ROOMS": "median"
}).reset_index()


# —— Step 7: 合并房产数据与代表性 EPC 特征 —— #
merged_df = pp_df.merge(
    epc_grouped,
    how="left",
    left_on=["postcode_clean", "ppd_property_type"],
    right_on=["POSTCODE_clean", "ppd_property_type"]
)

merged_df.to_csv("pp_epc_merged_2023.csv", index=False)

/var/folders/3f/_f93wsjd5qj0zgvg5mxm07yh0000gn/T/ipykernel_72657/618642486.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  epc_2023["POSTCODE_clean"] = epc_2023["POSTCODE"].str.replace(" ", "").str.upper()
/var/folders/3f/_f93wsjd5qj0zgvg5mxm07yh0000gn/T/ipykernel_72657/618642486.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  epc_2023["ppd_property_type"] = epc_2023.apply(map_property_type_epc, axis=1)


In [9]:

import numpy as np

# 读取数据
print("正在读取数据...")
df = pd.read_csv("pp_epc_merged_2023.csv")

print(f"原始数据形状: {df.shape}")
print(f"原始列名: {list(df.columns)}")

# 检查NA值情况
print("\n检查NA值情况:")
na_counts = df.isna().sum()
print(na_counts[na_counts > 0])

# 删除包含NA值的行
print(f"\n删除NA值前的行数: {len(df)}")
df_clean = df.dropna()
print(f"删除NA值后的行数: {len(df_clean)}")

# 检查transaction_id的重复情况
print(f"\n检查transaction_id重复情况:")
duplicate_counts = df_clean['transaction_id'].value_counts()
print(f"有重复的transaction_id数量: {(duplicate_counts > 1).sum()}")
print(f"重复最多的transaction_id出现次数: {duplicate_counts.max()}")

# 对于重复的transaction_id，保留第一个（或者可以根据需要选择其他策略）
df_clean = df_clean.drop_duplicates(subset=['transaction_id'], keep='first')
print(f"去重后的行数: {len(df_clean)}")

# 筛选ppd_category_type和record_status为A的记录
print(f"\n筛选前的行数: {len(df_clean)}")
df_clean = df_clean[(df_clean['ppd_category_type'] == 'A') & (df_clean['record_status'] == 'A')]
print(f"筛选后的行数: {len(df_clean)}")

# 去除price的top 1%和bottom 1%
print(f"\n去除price异常值前的行数: {len(df_clean)}")
print(f"Price统计信息:")
print(df_clean['price'].describe())

# 计算price的1%和99%分位数
price_1_percentile = df_clean['price'].quantile(0.01)
price_99_percentile = df_clean['price'].quantile(0.99)

print(f"Price 1%分位数: {price_1_percentile:,.0f}")
print(f"Price 99%分位数: {price_99_percentile:,.0f}")

# 筛选price在1%-99%分位数之间的数据
df_clean = df_clean[(df_clean['price'] >= price_1_percentile) & (df_clean['price'] <= price_99_percentile)]
print(f"去除price异常值后的行数: {len(df_clean)}")

# 定义新的列名映射（删除不需要的列）
column_mapping = {
    'transaction_id': 'transaction_id',
    'price': 'price',
    'date_of_transfer': 'date_of_transfer',
    'postcode': 'postcode',
    'property_type': 'property_type',
    'tenure_type': 'tenure_type',
    'county': 'county',
    'LAT': 'latitude',
    'LONG': 'longitude',
    'LSOA11': 'lsoa_code',
    'OA11': 'oa_code',
    'IMD': 'imd_score',
    'CTRY': 'country_code',
    'RGN': 'region_code',
    'OSLAUA': 'local_authority_code',
    'TOTAL_FLOOR_AREA': 'total_floor_area',
    'CONSTRUCTION_AGE_BAND': 'construction_age_band',
    'NUMBER_HABITABLE_ROOMS': 'number_habitable_rooms'
}

# 选择需要的列并重命名
columns_to_keep = list(column_mapping.keys())
available_columns = [col for col in columns_to_keep if col in df_clean.columns]

print(f"\n可用的列: {available_columns}")

# 选择需要的列
df_final = df_clean[available_columns].copy()

# 重命名列
df_final = df_final.rename(columns=column_mapping)

print(f"\n最终数据形状: {df_final.shape}")
print(f"最终列名: {list(df_final.columns)}")

# 保存清理后的数据
output_file = "pp_epc_merged_2023_clean.csv"
df_final.to_csv(output_file, index=False)
print(f"\n清理后的数据已保存到: {output_file}")

# 显示前几行数据
print("\n前5行数据:")
print(df_final.head())

# 显示数据基本信息
print("\n数据基本信息:")
print(df_final.info()) 

正在读取数据...
原始数据形状: (94835, 25)
原始列名: ['transaction_id', 'price', 'date_of_transfer', 'postcode', 'property_type', 'new_build_flag', 'tenure_type', 'county', 'ppd_category_type', 'record_status', 'PCDS', 'LAT', 'LONG', 'LSOA11', 'OA11', 'IMD', 'CTRY', 'RGN', 'OSLAUA', 'postcode_clean', 'ppd_property_type', 'POSTCODE_clean', 'TOTAL_FLOOR_AREA', 'CONSTRUCTION_AGE_BAND', 'NUMBER_HABITABLE_ROOMS']

检查NA值情况:
postcode                    233
PCDS                        247
LAT                         247
LONG                        247
LSOA11                      247
OA11                        247
IMD                         247
CTRY                        247
RGN                         247
OSLAUA                      247
postcode_clean              233
POSTCODE_clean            41569
TOTAL_FLOOR_AREA          41569
CONSTRUCTION_AGE_BAND     41569
NUMBER_HABITABLE_ROOMS    46712
dtype: int64

删除NA值前的行数: 94835
删除NA值后的行数: 48123

检查transaction_id重复情况:
有重复的transaction_id数量: 0
重复最多的transaction_id出